In [119]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2 as cv2

In [174]:
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [50]:

import csv
from shutil import copyfile
import glob
import re
numbers = re.compile(r'(\d+)')

#read images
mrsPiggyImages = []
for infile in sorted(glob.glob('FolderFrames/AllShowsFULL/*.jpg'), key=numericalSort):
    img = cv2.imread(infile, cv2.IMREAD_UNCHANGED)
    mrsPiggyImages.append(img)
    

#read GT
mrsPiggyGT = []
with open('./FolderFrames/GT_AllShows.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            #print(f'Column names are {", ".join(row)}')
            line_count += 1
        else: 
            framenumber = int(row[0])
            hasKermit = int(row[1])
            hasMrsPiggy = int(row[2])
            if hasMrsPiggy:
                mrsPiggyGT.append(np.asarray(1))
            else:
                mrsPiggyGT.append(np.asarray(0))
            line_count += 1
        
        #print(f'Processed {line_count} lines.')
    
mrsPiggyGT = np.array(mrsPiggyGT)


In [167]:
newImgNPArray = []
for image in mrsPiggyImages:
    npIMG = np.array(image)
    npIMG = cv2.resize(npIMG, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    #npIMG = np.resize(npIMG, (-1, (64, 64)))
    npIMG = np.expand_dims(npIMG, 3)
    newImgNPArray.append(npIMG)
len(newImgNPArray)

4500

In [168]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


X_train, X_test, y_train, y_test = train_test_split(newImgNPArray, mrsPiggyGT, test_size=0.1)


print(len(X_train))
print(len(y_train))

def build_model():
    model = Sequential()
    # Convolution - extracting appropriate features from the input image.
    # Non-Linearity (RELU) - replacing all negative pixel values in feature map by zero.
    
    model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3),
                   activation='relu'))
    # Pooling: reduces dimensionality of the feature maps but keeps the most important information.
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Adding a second convolutional layer and flattening in order to arrange 3D volumes into a 1D vector.
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    # Fully connected layers: ensures connections to all activations in the previous layer.
    model.add(Dense(units=128, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    return model


keras_model = build_model()
keras_model.compile(optimizer='adam', loss='binary_crossentropy',
                   metrics=['accuracy'])
keras_model.fit(X_train, y_train, epochs=81,  batch_size=50, verbose=1)

from sklearn.metrics import roc_curve
y_pred_keras = keras_model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

4050
4050


ValueError: Error when checking input: expected conv2d_88_input to have shape (64, 64, 3) but got array with shape (64, 3, 1)

In [92]:
X2, y2 = make_classification(n_samples=80000)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.5)

In [169]:
#print(str(list(X_train2)[0]))
print("x_trainSHOULDBE shape: ", np.array(X_train2).shape)
print("x_trainIS shape: ", np.array(X_train).shape)

print("X_SHOULDBE: ", X2.shape)
print("X_IS: ", np.array(newImgNPArray).shape)


x_trainSHOULDBE shape:  (40000, 20)
x_trainIS shape:  (4050, 64, 64, 3, 1)
X_SHOULDBE:  (80000, 20)
X_IS:  (4500, 64, 64, 3, 1)


In [171]:
train_imagedata = ImageDataGenerator(rescale=1. / 255, shear_range=0.2,
        zoom_range=0.2, horizontal_flip=True)
training_set = \
    train_imagedata.flow_from_directory('./FolderFrames/MrsPiggyDataset/MrsPiggyTrainingSet'
        , target_size=(64, 64), batch_size=12, class_mode='binary')

Found 3642 images belonging to 2 classes.


In [173]:
training_set